In [1]:
PROJECT_ID = "formazione-riccardo-zanella"
REGION = 'us-central1'
BUCKET_NAME = "bbs-2021-opml4b-explainability"

In [12]:
EXPORT_PATH = 'gs://' + BUCKET_NAME + '/mdl/image_data/image_data_20211103_112738/model/'
# EXPORT_PATH = '../../mdl/image_data/image_data_20211103_112738/model/'
MODEL = 'flowers'

# Model load

In [6]:
import tensorflow as tf
import numpy as np

In [7]:
model = tf.keras.models.load_model(EXPORT_PATH)
# Print the names of your tensors
print('Model input tensor: ', model.input.name)
print('Model output tensor: ', model.output.name)

Model input tensor:  conv2d_input
Model output tensor:  dense/Softmax:0


## Check for all needed signatures

In [8]:
input_name = list(model.signatures['xai_model'].structured_input_signature[1].keys())[0]
print(input_name)

output_name = list(model.signatures['xai_model'].structured_outputs.keys())[0]
print(output_name)

preprocess_name = list(model.signatures['xai_preprocess'].structured_input_signature[1].keys())[0]
print(preprocess_name)

numpy_inputs
output_0
bytes_inputs


In [9]:
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder
# We want to explain 'xai_model' signature.
builder = SavedModelMetadataBuilder(EXPORT_PATH, signature_name='xai_model')
random_baseline = np.random.rand(192, 192, 3)
builder.set_image_metadata(
    'numpy_inputs',
    input_baselines=[random_baseline.tolist()])
builder.save_metadata(EXPORT_PATH)

# Model deploy

In [10]:
! gcloud ai-platform models create $MODEL --enable-logging --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/formazione-riccardo-zanella/models/flowers].


To take a quick anonymous survey, run:
  $ gcloud survey



In [11]:
VERSION = 'v1'

In [13]:
! gcloud beta ai-platform versions create $VERSION --region=$REGION \
--model $MODEL \
--origin $EXPORT_PATH \
--runtime-version 2.4 \
--framework TENSORFLOW \
--python-version 3.7 \
--machine-type n1-standard-4

Using endpoint [https://us-central1-ml.googleapis.com/]
Creating version (this might take a few minutes)......done.
